In [2]:
import pandas as pd
import requests
from tqdm import tqdm
import torch
import esm
import pandas as pd
import numpy as np
import os
from transformers import AutoTokenizer, AutoModel
#import deepchem as dc
#import wandb



In [2]:
# 加载数据
luo_protein_name = pd.read_csv('/data/zyf/HyperGCN-DTI/data/Luo/protein_seq.csv')
Zheng_protein_name = pd.read_csv('/data/zyf/HyperGCN-DTI/data/Zheng/protein_seq.csv')
luo_drug_name =  pd.read_csv('/data/zyf/HyperGCN-DTI/data/Luo/protein_seq.csv')


In [20]:

if __name__ == "__main__":
    os.chdir('/data/zyf/HyperGCN-DTI/data')
    for csv_path in ['./Yamanishi/protein_seq.csv','./Zheng/protein_seq.csv']:
        extractor = ProteinFeatureExtractor(model_name='esm2_t6_8M_UR50D')
        # 从 CSV 文件中提取特征并保存
        extractor.extract_features_from_csv(csv_path)


Loaded model with 6 layers.


Processing sequences: 100%|██████████| 989/989 [03:52<00:00,  4.26it/s]


Features successfully extracted and saved to: ./Yamanishi/protein_seq.csv
Loaded model with 6 layers.


Processing sequences: 100%|██████████| 1556/1556 [06:04<00:00,  4.27it/s]


Features successfully extracted and saved to: ./Zheng/protein_seq.csv


In [21]:
luo_drug_name = pd.read_csv('/data/zyf/HyperGCN-DTI/data/Luo/drug.txt', header=None, names=['DrugBank ID'])
Zheng_drug_name = pd.read_csv('/data/zyf/HyperGCN-DTI/data/Zheng/drug_dic.txt', header=None, names=['DrugBank ID'])
mapping_data = pd.read_csv('/data/zyf/HyperGCN-DTI/data/drugbank/BioDb/drugbank/drugbank_small_molecule.csv')
data = mapping_data[['drugbank_id','smiles']]
data.columns = ['DrugBank ID',"SMILES"]
luo_drug_smiles = get_smiles(luo_drug_name,data)
zheng_drug_smiles = get_smiles(Zheng_drug_name,data)
luo_drug_smiles.to_csv('/data/zyf/HyperGCN-DTI/data/Luo/drug_smiles.csv', index=False)
zheng_drug_smiles.to_csv('/data/zyf/HyperGCN-DTI/data/Zheng/drug_smiles.csv', index=False)


Number of missing values in 'SMILES' column: 4
Number of missing values in 'SMILES' column: 13


In [2]:
from feature_fusion import SMILESFeatureExtractor

# 使用示例
if __name__ == "__main__":
    csv_path = '/data/zyf/HyperGCN-DTI/data/Zheng/drug_smiles.csv'  
    extractor = SMILESFeatureExtractor(model_name='DeepChem/ChemBERTa-77M-MTR')
    extractor.process_and_save_features(csv_path)


Some weights of RobertaModel were not initialized from the model checkpoint at /data/zyf/HyperGCN-DTI/codes/ChemBert and are newly initialized: ['roberta.pooler.dense.bias', 'roberta.pooler.dense.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


Processing SMILES: 100%|██████████| 1094/1094 [00:02<00:00, 399.11it/s]


Features successfully extracted and saved to: /data/zyf/HyperGCN-DTI/data/Zheng/drug_smiles.csv


In [61]:
class KEGGDrugScraper:
    def __init__(self, input_csv_path='/data/zyf/HyperGCN-DTI/data/Yamanishi/drug_smiles.csv',
                        output_csv_path='/data/zyf/HyperGCN-DTI/data/Yamanishi/drug_disease.csv'):
        """
        初始化 KEGGDrugScraper 类

        Parameters:
        - kegg_ids: list
            KEGG Drug ID 列表
        - output_csv_path: str, optional
            保存 CSV 文件的路径，默认路径为 '/mnt/data/kegg_drug_info.csv'
        """
        self.drug_name = pd.read_csv(input_csv_path)
        self.kegg_ids =self.drug_name['KEGG Drug ID'].dropna().unique().tolist()
        self.output_csv_path = output_csv_path
        self.results = []
    
    def fetch_kegg_info(self, kegg_id):
        """
        根据 KEGG Drug ID 获取 Efficacy 和 Disease 信息，并将它们合并
        """
        url = f"https://www.genome.jp/dbget-bin/www_bget?dr:{kegg_id}"
        response = requests.get(url)
        
        if response.status_code == 200:
            soup = BeautifulSoup(response.content, 'html.parser')
            
            # 查找 Efficacy 信息
            efficacy_section = soup.find(string="Efficacy")
            if efficacy_section:
                efficacy = efficacy_section.find_next('td').get_text(strip=True, separator='; ')
            else:
                efficacy = 'N/A'
            
            # 查找 Disease 信息
            disease_section = soup.find(string="Disease")
            if disease_section:
                disease_entries = disease_section.find_next('td').find_all('a')
                disease_list = [entry.get_text(strip=True) for entry in disease_entries]
                disease = '; '.join(disease_list)
            else:
                disease = 'N/A'
            
            # 将 Efficacy 和 Disease 合并
            efficacy_disease = f"{efficacy}; {disease}" if efficacy != 'N/A' and disease != 'N/A' else efficacy if disease == 'N/A' else disease
            
            return efficacy_disease
        else:
            return 'N/A'
    
    def scrape_data(self):
        """
        遍历 KEGG Drug ID 列表并爬取信息
        """
        for kegg_id in tqdm(self.kegg_ids):
            #print(f"Fetching data for KEGG Drug ID: {kegg_id}")
            efficacy_disease = self.fetch_kegg_info(kegg_id)
            #print(f"Efficacy and Disease: {efficacy_disease}")
            
            self.results.append({'KEGG Drug ID': kegg_id, 'Efficacy and Disease': efficacy_disease})
            
            # 等待一会儿以避免对服务器造成压力
            time.sleep(1)
    
    def save_to_csv(self):
        """
        将爬取结果保存到 CSV 文件
        """
        df = pd.DataFrame(self.results)
        df.to_csv(self.output_csv_path, index=False)
        print(f"Data has been saved to {self.output_csv_path}")
    
    def run(self):
        """
        执行完整的爬取和保存流程
        """
        self.scrape_data()
        self.save_to_csv()


scraper = KEGGDrugScraper()
scraper.run()



100%|██████████| 791/791 [34:35<00:00,  2.62s/it]

Data has been saved to /data/zyf/HyperGCN-DTI/data/Yamanishi/drug_disease.csv


In [62]:
import requests
import pandas as pd
import time

def get_uniprot_id(prot_id):
    """Fetch the UniProt ID from KEGG using the provided prot_id."""
    url = f"http://rest.kegg.jp/get/hsa:{prot_id}"
    response = requests.get(url)
    
    if response.status_code == 200:
        # Find the line containing the UniProt ID
        for line in response.text.splitlines():
            if "UniProt" in line:
                # Extract the UniProt ID
                uniprot_id = line.split()[1]
                return uniprot_id
    return None

# List of prot_ids to query
prot_ids = ['10', '100', '10056', '1017', '1018', '10188', '1019', '1020', '1021', '1022', '1024', '1025', '10269', '10279', '10295']

# Create a DataFrame to store the results
results = []

for prot_id in prot_ids:
    uniprot_id = get_uniprot_id(prot_id)
    results.append({'prot_id': prot_id, 'UniProt_ID': uniprot_id})
    
    # Sleep to avoid overwhelming the KEGG server
    time.sleep(1)

# Convert results to DataFrame
df = pd.DataFrame(results)

# Save the DataFrame to a CSV file
df.to_csv('/data/zyf/HyperGCN-DTI/data/prot_id_to_uniprot_id.csv', index=False)

print("The UniProt_IDs have been successfully saved to prot_id_to_uniprot_id.csv")


The UniProt_IDs have been successfully saved to prot_id_to_uniprot_id.csv


In [10]:
data = pd.read_csv('/data/zyf/HyperGCN-DTI/data/drugbank/BioDb/drugbank/drugbank_small_molecule.csv')

In [11]:
data.tail()

,type,drugbank_id,name,description,indication,unii,cas-num,groups,atc_codes,categories,inchi,inchikey,smiles,aliases
2664,small molecule,DB15934,Ferric glycerophosphate,NaN,NaN,MUL7ONL3Z6,1301-70-8,approved|experimental,NaN,NaN,InChI=1S/C5H13O6P.C3H9O6P.2Fe/c6-3-1-5(2-4-7)1...,CBEYZWBXIPZYNH-UHFFFAOYSA-J,[Fe++].[Fe++].OCC(O)COP([O-])([O-])=O.OCCC(CCO...,Ferric glycerophosphate|Multiple Mineral Liquid
2665,small molecule,DB15950,Sodium camphorsulfonate,NaN,NaN,CFN2D8XEIU,21791-94-6,approved|experimental,NaN,NaN,"InChI=1S/C10H16O4S.Na/c1-9(2)7-3-4-10(9,8(11)5...",AWMAOFAHBPCBHJ-YZUKSGEXSA-M,[Na+].[H][C@]12CC[C@@](CS([O-])(=O)=O)(C(=O)C1...,Broncho Tuss Adult|Broncho Tuss Infant|Sodium ...
2666,small molecule,DB15982,Berotralstat,Berotralstat is a selective inhibitor of plasm...,Berotralstat is indicated for prophylaxis of a...,XZA0KB1BDQ,1809010-50-1,approved,NaN,BCRP/ABCG2 Substrates|Cytochrome P-450 CYP2D6 ...,InChI=1S/C30H26F4N6O/c31-24-10-9-22(28(37-17-1...,UXNXMBYCBRBRFD-MUUNZHRXSA-N,NCC1=CC(=CC=C1)N1N=C(C=C1C(=O)NC1=CC(=CC=C1F)[...,Berotralstat|Orladeyo
2667,small molecule,DB16019,Gallium Ga-68 gozetotide,Gallium (Ga) 68 prostate-specific membrane ant...,"Gallium Ga-68 gozetotide, or Gallium Ga-68 PSM...",ZJ0EKR6M10,1906894-20-9,approved,NaN,"Acetates|Acids, Acyclic|Amines|Amino Acids, Pe...",InChI=1S/C44H62N6O17.Ga/c51-34-13-8-28(22-30(3...,AEBYHKKMCWUMKX-LNTZDJBBSA-K,[H+].[68Ga+3].OC(=O)CC[C@H](NC(=O)N[C@@H](CCCC...,\n |Gallium Ga-68 gozetotide
2668,small molecule,DB16022,Thenyldiamine,NaN,NaN,E4U52363JE,91-79-2,approved|experimental,NaN,NaN,InChI=1S/C14H19N3S/c1-16(2)8-9-17(11-13-6-10-1...,RCGYDFVCAAKKNG-UHFFFAOYSA-N,CN(C)CCN(CC1=CSC=C1)C1=CC=CC=N1,Neo Synephrine Nasal Spray|Neo-synephrine Nasa...


In [17]:
luo_drug_features = luo_drug_name.merge(data, on='DrugBank ID', how='left')

# Display the resulting dataframe containing the matching SMILES
luo_drug_features = luo_drug_features[['DrugBank ID', 'SMILES']]
# Drop duplicate rows based on the 'DrugBank ID' column
luo_drug_features = luo_drug_features.drop_duplicates(subset='DrugBank ID', keep='first')

# Check for any missing values in the 'SMILES' column
missing_values = luo_drug_features['SMILES'].isna().sum()

# Display the cleaned data and the count of missing values
print(f"Number of missing values in 'SMILES' column: {missing_values}")
print(luo_drug_features)


Number of missing values in 'SMILES' column: 45
     DrugBank ID                                             SMILES
0        DB00050  Clc1ccc(C[C@@H](NC(=O)[C@H](NC(=O)C)Cc2cc3c(cc...
2        DB00152                  OCCc1c(C)[n+](Cc2c(N)nc(C)nc2)cs1
4        DB00162       OC/C=C(/C=C/C=C(/C=C/C=1C(C)(C)CCCC=1C)\C)\C
20       DB00175  O=C(O[C@@H]1[C@@H]2[C@@H](CC[C@@H](O)C[C@@H](O...
21       DB00176                  FC(F)(F)c1ccc(C(=NOCCN)CCCCOC)cc1
...          ...                                                ...
3392     DB06711                        C(c1c2c(ccc1)cccc2)C1=NCCN1
3394     DB06800  O=C1[C@@H]2Oc3c(O)ccc4c3[C@@]32[C@](O)([C@H]([...
3396     DB06811                                                NaN
3397     DB08826                                                NaN
3398     DB08860  Fc1ccc(-c2c(/C=C/[C@@H](O)C[C@@H](O)CC(=O)O)c(...

[708 rows x 2 columns]


In [14]:
luo_drug_name = pd.read_csv('/data/zyf/HyperGCN-DTI/data/Luo/drug.txt', header=None, names=['DrugBank ID'])
Zheng_drug_name = pd.read_csv('/data/zyf/HyperGCN-DTI/data/Zheng/drug_dic.txt', header=None, names=['DrugBank ID'])
extracted_data = data[['mol', 'BindingDB Reactant_set_id', 'DrugBank ID of Ligand', 'KEGG ID of Ligand']]
# Extracting data
extracted_data = data[['mol', 'BindingDB Reactant_set_id', 'DrugBank ID of Ligand', 'KEGG ID of Ligand']]
extracted_data.columns = ['SMILES', 'BindingDB ID', 'DrugBank ID', 'KEGG Drug ID']


In [45]:
extracted_data.head()

,SMILES,BindingDB ID,DrugBank ID,KEGG Drug ID
0,O=C([C@@H](O)[C@@H](NC(=O)c1c(C)c(O)ccc1)Cc1cc...,842,NaN,NaN
1,O=C([C@@H](O)[C@@H](NC(=O)c1c(C)c(O)ccc1)Cc1cc...,852,NaN,NaN
2,O=C(OCc1ccccc1)N[C@H](C(=O)N[C@H]([C@@H](O)[C@...,1061,NaN,NaN
3,O=C(OCc1ccccc1)N[C@H](C(=O)N[C@H]([C@@H](O)[C@...,1083,NaN,NaN
4,S(=O)(=O)(Nc1cc(C(CC)C2C(=O)OC(CCC)(CCc3ccccc3...,1134,NaN,NaN


In [65]:
import requests
from bs4 import BeautifulSoup

# Start a session
session = requests.Session()

# Headers to mimic a real browser
headers = {
    "User-Agent": "Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/58.0.3029.110 Safari/537.3"
}

# Step 1: Access the login page to fetch the authenticity token
login_url = "https://go.drugbank.com/users/sign_in"
login_page = session.get(login_url, headers=headers)

# Check the returned content
print(login_page.text)  # Debug: Output the HTML to see if it matches expectations

# Parse the login page
soup = BeautifulSoup(login_page.text, 'html.parser')

# Extract the authenticity token
auth_token = soup.find('input', {'name': 'authenticity_token'})
if auth_token:
    auth_token = auth_token.get('value')
    print("Found authenticity_token:", auth_token)
else:
    print("Could not find authenticity_token. Please check the login page structure.")
    exit()


<!DOCTYPE html><html lang="en-US"><head><title>Just a moment...</title><meta http-equiv="Content-Type" content="text/html; charset=UTF-8"><meta http-equiv="X-UA-Compatible" content="IE=Edge"><meta name="robots" content="noindex,nofollow"><meta name="viewport" content="width=device-width,initial-scale=1"><style>*{box-sizing:border-box;margin:0;padding:0}html{line-height:1.15;-webkit-text-size-adjust:100%;color:#313131;font-family:system-ui,-apple-system,BlinkMacSystemFont,Segoe UI,Roboto,Helvetica Neue,Arial,Noto Sans,sans-serif,Apple Color Emoji,Segoe UI Emoji,Segoe UI Symbol,Noto Color Emoji}body{display:flex;flex-direction:column;height:100vh;min-height:100vh}.main-content{margin:8rem auto;max-width:60rem;padding-left:1.5rem}@media (width <= 720px){.main-content{margin-top:4rem}}.h2{font-size:1.5rem;font-weight:500;line-height:2.25rem}@media (width <= 720px){.h2{font-size:1.25rem;line-height:1.5rem}}#challenge-error-text{background-image:url();background-repeat:no-repeat;background-s

: 

In [48]:
luo_drug_features = luo_drug_name.merge(extracted_data, on='DrugBank ID', how='left')
# Display the resulting dataframe containing the matching SMILES
#luo_drug_features = luo_drug_features[['DrugBank ID', 'SMILES']]
# Drop duplicate rows based on the 'DrugBank ID' column
luo_drug_features = luo_drug_features.drop_duplicates(subset='DrugBank ID', keep='first')

# Check for any missing values in the 'SMILES' column
missing_values = luo_drug_features['BindingDB ID'].isna().sum()

# Display the cleaned data and the count of missing values
print(f"Number of missing values in 'BindingDB ID' column: {missing_values}")
print(luo_drug_features)

Number of missing values in 'BindingDB ID' column: 189
      DrugBank ID                                             SMILES  \
0         DB00050                                                NaN   
1         DB00152                  OCCc1c(C)[n+](Cc2c(N)nc(C)nc2)cs1   
14        DB00162       OC/C=C(/C=C/C=C(/C=C/C=1C(C)(C)CCCC=1C)\C)\C   
17        DB00175  O=C(O[C@@H]1[C@@H]2[C@@H](CC[C@@H](O)C[C@@H](O...   
40        DB00176                  FC(F)(F)c1ccc(C(=NOCCN)CCCCOC)cc1   
...           ...                                                ...   
39443     DB06711                        C(c1c2c(ccc1)cccc2)C1=NCCN1   
39476     DB06800  O=C1[C@@H]2Oc3c(O)ccc4c3[C@@]32[C@](O)([C@H]([...   
39496     DB06811                                                NaN   
39497     DB08826                                                NaN   
39498     DB08860  Fc1ccc(-c2c(/C=C/[C@@H](O)C[C@@H](O)CC(=O)O)c(...   

       BindingDB ID KEGG Drug ID  
0               NaN          NaN  
1         

In [49]:
ya_drug_name =  pd.read_csv('/data/zyf/HyperGCN-DTI/data/Yamanishi/drug_smiles.csv')

In [41]:
print(ya_drug_name.head())
extracted_data['KEGG Drug ID']

  KEGG Drug ID                                             SMILES  feature_1  \
0       D00448  OC(=O)C1=CC(=CC=C1O)\N=N\C1=CC=C(C=C1)S(=O)(=O...  -0.045468   
1       D00037                         OC(=O)CC(O)(CC(O)=O)C(O)=O   0.061610   
2       D00155    OC[C@H]1O[C@H](C[C@@H]1O)N1C=NC2=C1N=CNC[C@H]2O  -0.038069   
3       D00021                        N[C@@H](CC1=CC=CC=C1)C(O)=O   0.010332   
4       D01441  CN1CCN(CC2=CC=C(C=C2)C(=O)NC2=CC(NC3=NC=CC(=N3...  -0.044214   

   feature_2  feature_3  feature_4  feature_5  feature_6  feature_7  \
0  -0.005802  -0.073091   0.016811  -0.043807   0.044790  -0.082236   
1   0.148773  -0.106675  -0.051982   0.081267  -0.134128   0.057199   
2   0.020136  -0.046006  -0.017700   0.093686  -0.022143  -0.057902   
3  -0.005363  -0.127564   0.021989   0.024732  -0.098402  -0.045624   
4  -0.047543   0.113636   0.003911  -0.061161  -0.056651  -0.023843   

   feature_8  ...  feature_375  feature_376  feature_377  feature_378  \
0  -0.012501  ...  

0             NaN
1             NaN
2             NaN
3             NaN
4             NaN
            ...  
2785721    C07675
2785722       NaN
2785723       NaN
2785724    C15493
2785725       NaN
Name: KEGG Drug ID, Length: 2785726, dtype: object

In [43]:
ya_drug_features = ya_drug_name.merge(extracted_data[['BindingDB ID','KEGG Drug ID']], on='KEGG Drug ID', how='left')
ya_drug_features = ya_drug_features.drop_duplicates(subset='KEGG Drug ID', keep='first')
# Check for any missing values in the 'SMILES' column
missing_values = ya_drug_features['BindingDB ID'].isna().sum()

# Display the cleaned data and the count of missing values
print(f"Number of missing values in 'SMILES' column: {missing_values}")
print(ya_drug_features.head())

Number of missing values in 'SMILES' column: 791
  KEGG Drug ID                                             SMILES  feature_1  \
0       D00448  OC(=O)C1=CC(=CC=C1O)\N=N\C1=CC=C(C=C1)S(=O)(=O...  -0.045468   
1       D00037                         OC(=O)CC(O)(CC(O)=O)C(O)=O   0.061610   
2       D00155    OC[C@H]1O[C@H](C[C@@H]1O)N1C=NC2=C1N=CNC[C@H]2O  -0.038069   
3       D00021                        N[C@@H](CC1=CC=CC=C1)C(O)=O   0.010332   
4       D01441  CN1CCN(CC2=CC=C(C=C2)C(=O)NC2=CC(NC3=NC=CC(=N3...  -0.044214   

   feature_2  feature_3  feature_4  feature_5  feature_6  feature_7  \
0  -0.005802  -0.073091   0.016811  -0.043807   0.044790  -0.082236   
1   0.148773  -0.106675  -0.051982   0.081267  -0.134128   0.057199   
2   0.020136  -0.046006  -0.017700   0.093686  -0.022143  -0.057902   
3  -0.005363  -0.127564   0.021989   0.024732  -0.098402  -0.045624   
4  -0.047543   0.113636   0.003911  -0.061161  -0.056651  -0.023843   

   feature_8  ...  feature_376  feature_377